In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
train_df = pd.read_csv('/code/MyCode/AUG/HAM10000/train.csv')
test_df = pd.read_csv('/code/MyCode/AUG/HAM10000/test.csv')
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
train_path = '/code/MyCode/AUG/HAM10000/train_dir'
test_path = '/code/MyCode/AUG/HAM10000/test_dir'
batch_size=16

In [3]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

In [4]:
image_size = 299
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 55115 images belonging to 7 classes.

Test Batches: 
Found 2003 images belonging to 7 classes.


In [5]:
#Soft Attention

from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
import tensorflow as tf



class SoftAttention(Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x


        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC

        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]

        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape


        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = K.expand_dims(x,axis=-1)

        c3d = K.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = K.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = kl.Activation('relu')(conv3d)

        conv3d = K.permute_dimensions(conv3d,pattern=(0,4,1,2,3))


        conv3d = K.squeeze(conv3d, axis=-1)
        conv3d = K.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = K.softmax(conv3d, axis=-1)
        softmax_alpha = kl.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)


        if self.aggregate_channels==False:
            exp_softmax_alpha = K.expand_dims(softmax_alpha, axis=-1)
            exp_softmax_alpha = K.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))

            x_exp = K.expand_dims(x,axis=-2)

            u = kl.Multiply()([exp_softmax_alpha, x_exp])

            u = kl.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = K.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = K.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = K.expand_dims(exp_softmax_alpha, axis=-1)

            u = kl.Multiply()([exp_softmax_alpha, x])

        if self.concat_input_with_scaled:
            o = kl.Concatenate(axis=-1)([u,x])
        else:
            o = u

        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape):
        return [self.out_features_shape, self.out_attention_maps_shape]


    def get_config(self):
        return super(SoftAttention,self).get_config()


In [6]:

irv2 = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classifier_activation="softmax",

)

# Excluding the last 28 layers of the model.
conv = irv2.layers[-28].output


In [7]:

#SOFT ATTENTION
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv  = Activation('relu')(conv)
conv = Dropout(0.5)(conv)


output = Flatten()(conv)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=irv2.input, outputs=output)


In [8]:
from tensorflow.keras import models
model.load_weights("/code/MyCode/AUG/Aug-Att/IRV2+SA.hdf5")
predictions_IRV2 = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

DENSENET

In [9]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 55115 images belonging to 7 classes.

Test Batches: 
Found 2003 images belonging to 7 classes.


In [10]:
densenet = tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,

)
# Exclude the last 28 layers of the model.
conv = densenet.layers[-28].output

In [11]:

#SOFT ATTENTION
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv  = Activation('relu')(conv)
conv = Dropout(0.5)(conv)


output = Flatten()(conv)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=densenet.input, outputs=output)

In [12]:
from tensorflow.keras import models
model.load_weights("/code/MyCode/AUG/Aug-Att/Densenet201+SA.hdf5")
predictions_Densenet = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

resnet50

In [13]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 55115 images belonging to 7 classes.

Test Batches: 
Found 2003 images belonging to 7 classes.


In [14]:
resnet = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

# Exclude the last 3 layers of the model.
conv = resnet.layers[-3].output

In [15]:

attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv  = Activation('relu')(conv)
conv = Dropout(0.5)(conv)


In [16]:
from keras.layers import GlobalAveragePooling2D

output = GlobalAveragePooling2D()(conv)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=resnet.input, outputs=output)

In [17]:
from tensorflow.keras import models
model.load_weights("/code/MyCode/AUG/Aug-Att/ResNet50+SA.hdf5")
predictions_Resnet50 = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

resnet34

In [18]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 55115 images belonging to 7 classes.

Test Batches: 
Found 2003 images belonging to 7 classes.


In [19]:
MainInput=Input(shape=(224, 224, 3))

In [20]:
def convlayer1(input_value):
  conv1=Conv2D(filters=64, kernel_size=(3,3), strides=(2,2),activation="relu",padding="same")(input_value)
  conv1=BatchNormalization()(conv1)
  pool1=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)
  return pool1

In [21]:
def convlayer2(input_value):
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv2=BatchNormalization()(conv2)
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(conv2)
  conv2=BatchNormalization()(conv2)

  resnet=add([input_value,conv2])
  resnet=Activation("relu")(resnet)
  return resnet

In [22]:
def convlayer3(input_value):
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv3=BatchNormalization()(conv3)
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(conv3)
  conv3=BatchNormalization()(conv3)


  skip=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv3])
  resnet=Activation("relu")(resnet)
  return resnet

In [23]:
def convlayer4(input_value):
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv4=BatchNormalization()(conv4)
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(conv4)
  conv4=BatchNormalization()(conv4)


  skip=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv4])
  resnet=Activation("relu")(resnet)
  return resnet

In [24]:
def convlayer5(input_value):
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv5=BatchNormalization()(conv5)
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(conv5)
  conv5=BatchNormalization()(conv5)


  skip=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv5])
  resnet=Activation("relu")(resnet)
  return resnet

In [25]:
block1=convlayer1(MainInput)

In [26]:
block2=convlayer2(block1)
for i in range(0,2):
  block2=convlayer2(block2)

In [27]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block2)
block3=convlayer3(maxpool)

for i in range(0,3):
  block3=convlayer3(block3)

In [28]:

attention_layer2,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(block3.shape[-1]),name='soft_attention')(block3)
attention_layer2=MaxPooling2D(pool_size=(2,2), padding='same')(attention_layer2)
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block3)

concat2=concatenate([maxpool,attention_layer2])
conv = Activation("relu")(concat2)
conv= Dropout(0.5)(conv)

In [29]:
block4=convlayer4(conv)
for i in range(0,5):
  block4=convlayer4(block4)

In [30]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block4)
block5=convlayer5(maxpool)
for i in range(0,2):
  block5=convlayer5(block5)

In [31]:

output = GlobalAveragePooling2D()(block5)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=MainInput, outputs=output)

In [32]:
from tensorflow.keras import models
model.load_weights("/code/MyCode/AUG/Aug-Att/ResNet34+SA.hdf5")
predictions_Resnet34 = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

vgg16

In [33]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 55115 images belonging to 7 classes.

Test Batches: 
Found 2003 images belonging to 7 classes.


In [34]:
MainInput=Input(shape=(224, 224, 3))

In [35]:
conv=(Conv2D(filters=64,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(MainInput))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=64,kernel_size=(1,1), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))

In [36]:
conv=(MaxPooling2D(strides=(2, 2),padding="same")(conv))

In [37]:
conv=(Conv2D(filters=128,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=128,kernel_size=(1,1), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))

In [38]:
conv=(MaxPooling2D()(conv))

In [39]:
conv=(Conv2D(filters=256,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=256,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=256,kernel_size=(1,1), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))

In [40]:
conv=(MaxPooling2D()(conv))

In [41]:
conv=(Conv2D(filters=512,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=512,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=512,kernel_size=(1,1), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))

In [42]:
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(conv.shape[-1]),name='soft_attention')(conv)
attention_layer=(MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer))
conv=(MaxPooling2D(pool_size=(2, 2),padding="same")(conv))

conv = concatenate([conv,attention_layer])
conv=Activation("relu")(conv)
conv= Dropout(0.5)(conv)

In [43]:
conv=(Conv2D(filters=512,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=512,kernel_size=(3,3), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))
conv=(Conv2D(filters=512,kernel_size=(1,1), activation="relu",padding="same",kernel_initializer='he_normal')(conv))
conv=(BatchNormalization()(conv))

In [44]:
conv=(MaxPooling2D(pool_size=(4, 4),padding="same")(conv))

In [45]:
conv=(Flatten()(conv))
conv=(Dense(4096,activation="relu")(conv))
conv=(Dense(4096,activation="relu")(conv))
conv=(Dense(7, activation="softmax")(conv))

model = Model(inputs=MainInput, outputs=conv)

In [46]:
from tensorflow.keras import models
model.load_weights("/code/MyCode/AUG/Aug-Att/vgg16+SA.hdf5")
predictions_VGG16 = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
predictions_IRV2      = np.load("predictions_IRV2.npy")
predictions_Densenet  = np.load("predictions_Densenet.npy")
predictions_Resnet50  = np.load("predictions_Resnet50.npy")
predictions_Resnet34  = np.load("predictions_Resnet34.npy")
predictions_VGG16     = np.load("predictions_VGG16.npy")

y_true = np.load("y_true.npy")

targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

all_preds = [predictions_IRV2, predictions_Densenet, predictions_Resnet50,
             predictions_Resnet34, predictions_VGG16]

N, C = predictions_IRV2.shape


In [ ]:
def normalize_mass(m):
    s = np.sum(m)
    return m / s if s != 0 else m

def ds_combine_two(m1, m2):
    C = m1.shape[0]
    m_comb = np.zeros(C)
    conflict = 0.0

    for i in range(C):
        for j in range(C):
            if i == j:
                m_comb[i] += m1[i] * m2[j]
            else:
                conflict += m1[i] * m2[j]

    if conflict < 1.0:
        m_comb = m_comb / (1.0 - conflict)
    else:
        m_comb = (m1 + m2) / 2.0

    return normalize_mass(m_comb)

def ds_fusion(predictions_list):
    fused = predictions_list[0].copy()
    for preds in predictions_list[1:]:
        for i in range(fused.shape[0]):
            fused[i] = ds_combine_two(fused[i], preds[i])
    return fused


In [ ]:
teacher_ds = ds_fusion(all_preds)


In [ ]:
all_preds_array = np.stack(all_preds, axis=0)  
disagreement = np.std(all_preds_array, axis=0).mean(axis=1)  

threshold = disagreement.mean() + disagreement.std()
print(f"Disagreement threshold: {threshold:.4f}")


In [ ]:
def bayesian_update(prior, predictions_list, eps=1e-8):
   
    log_post = np.log(prior + eps)
    for pred in predictions_list:
        log_post += np.log(pred + eps)
    posterior = np.exp(log_post)
    posterior /= posterior.sum()
    return posterior


In [ ]:
teacher_hybrid = np.zeros_like(teacher_ds)

for i in range(N):
    if disagreement[i] <= threshold:
        # Low disagreement → keep DS output
        teacher_hybrid[i] = teacher_ds[i]
    else:
        # High disagreement → use DS as prior + Bayesian update
        sample_preds = [pred[i] for pred in all_preds]
        teacher_hybrid[i] = bayesian_update(teacher_ds[i], sample_preds)


In [ ]:
np.save("teacher_hybrid_probs.npy", teacher_hybrid)


In [ ]:
y_pred_hybrid = np.argmax(teacher_hybrid, axis=1)

print(classification_report(y_true, y_pred_hybrid, target_names=targetnames))


cm = confusion_matrix(y_true, y_pred_hybrid)

plt.figure(figsize=(8,6))
plt.imshow(cm)
plt.title("Hybrid DS + Bayesian Teacher – Confusion Matrix")
plt.colorbar()
plt.xticks(range(7), targetnames, rotation=45)
plt.yticks(range(7), targetnames)

thresh_cm = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j],
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh_cm else "black")

plt.tight_layout()
plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.show()


In [ ]:
def normalize_mass_map(m):
    s = np.sum(m)
    return m / s if s != 0 else m

def ds_combine_two_map(m1, m2):
    """
    Apply DS fusion on 3D or 4D importance maps (H,W,C)
    """
    m_comb = np.zeros_like(m1)
    conflict = 0.0
    
    flat1 = m1.flatten()
    flat2 = m2.flatten()
    
    for i in range(len(flat1)):
        for j in range(len(flat2)):
            if i == j:
                m_comb.flat[i] += flat1[i] * flat2[j]
            else:
                conflict += flat1[i] * flat2[j]

    if conflict < 1.0:
        m_comb = m_comb / (1.0 - conflict)
    else:
        m_comb = (m1 + m2) / 2.0
    
    return normalize_mass_map(m_comb)

def ds_fusion_map(lrp_list):
    fused = lrp_list[0].copy()
    for lrp in lrp_list[1:]:
        for i in range(fused.shape[0]):
            fused[i] = ds_combine_two_map(fused[i], lrp[i])
    return fused


In [ ]:
lrp_ds = ds_fusion_map(all_lrp)
print("LRP DS fusion done, shape:", lrp_ds.shape)


In [ ]:
all_lrp_array = np.stack(all_lrp, axis=0)  # shape: (num_models, N, H, W, C)
disagreement = np.std(all_lrp_array, axis=0).mean(axis=(1,2,3))  # mean over H,W,C

threshold = disagreement.mean() + disagreement.std()
print(f"LRP disagreement threshold: {threshold:.4f}")


In [ ]:
def bayesian_update_map(prior_map, maps_list, eps=1e-8):
    log_post = np.log(prior_map + eps)
    for m in maps_list:
        log_post += np.log(m + eps)
    posterior = np.exp(log_post)
    posterior /= posterior.sum()
    return posterior


In [ ]:
lrp_hybrid = np.zeros_like(lrp_ds)

for i in range(N):
    if disagreement[i] <= threshold:
        # low disagreement → keep DS map
        lrp_hybrid[i] = lrp_ds[i]
    else:
        # high disagreement → use DS as prior + Bayesian update
        sample_maps = [lrp[i] for lrp in all_lrp]
        lrp_hybrid[i] = bayesian_update_map(lrp_ds[i], sample_maps)


In [ ]:
np.save("LRP_hybrid.npy", lrp_hybrid)
print("Saved LRP_hybrid.npy with shape:", lrp_hybrid.shape)


In [ ]:
test_path = '/code/MyCode/AUG/HAM10000/test_dir'

test_batches = datagen.flow_from_directory(
    directory=test_path,
    target_size=input_shape[:2],
    batch_size=batch_size,
    shuffle=False
)

steps_test = len(test_batches)


In [ ]:
predictions_teacher = model.predict(test_batches, steps=steps_test, verbose=1)
np.save("teacher_predictions.npy", predictions_teacher)


In [ ]:
teacher_lrp_test = []
test_batches.reset()

for i in range(steps_test):
    X_batch, _ = test_batches.next()
    lrp_batch = compute_lrp(model, X_batch)  # LRP-0 for teacher
    teacher_lrp_test.append(lrp_batch)

teacher_lrp_test = np.vstack(teacher_lrp_test)
np.save("teacher_LRP.npy", teacher_lrp_test)


In [ ]:
import matplotlib.pyplot as plt

X_batch, _ = test_batches.next()
idx = 0

img = X_batch[idx]
lrp = teacher_lrp_test[idx].squeeze()

fig, axes = plt.subplots(1,2,figsize=(8,4))
axes[0].imshow((img - img.min()) / (img.max() - img.min()))
axes[0].set_title("Original Image")
axes[0].axis('off')

axes[1].imshow(lrp, cmap='hot')
axes[1].set_title("Teacher LRP")
axes[1].axis('off')

plt.show()


In [ ]:
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap

In [ ]:
class_names = ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']
n_classes = len(class_names)

teacher_hybrid = np.load("teacher_hybrid_probs.npy")  
n_samples = teacher_hybrid.shape[0]

predictions_IRV2      = np.load("predictions_IRV2.npy")
predictions_Densenet  = np.load("predictions_Densenet.npy")
predictions_Resnet50  = np.load("predictions_Resnet50.npy")
predictions_Resnet34  = np.load("predictions_Resnet34.npy")
predictions_VGG16     = np.load("predictions_VGG16.npy")

all_preds = [predictions_IRV2, predictions_Densenet, predictions_Resnet50,
             predictions_Resnet34, predictions_VGG16]

def normalize_predictions(pred):
    s = pred.sum(axis=1, keepdims=True)
    s[s==0] = 1
    return pred / s

teacher_hybrid = normalize_predictions(teacher_hybrid)
ensemble_mean = np.mean(all_preds, axis=0)  
ensemble_std = np.std(all_preds, axis=0)    


predicted_classes = np.argmax(np.stack(all_preds, axis=0), axis=2)
class_disagreement = np.mean(predicted_classes != predicted_classes[0], axis=0)

# t-SNE 
all_predictions = np.concatenate(all_preds, axis=1)  

feature_sets = {
    'raw_predictions': all_predictions,
    'ensemble_stats': np.concatenate([ensemble_mean, ensemble_std], axis=1),
    'disagreement_features': np.concatenate([ensemble_mean, ensemble_std, class_disagreement.reshape(-1,1)], axis=1),
    'pca_reduced': None
}

pca = PCA(n_components=min(50, all_predictions.shape[1]))
feature_sets['pca_reduced'] = pca.fit_transform(all_predictions)

features = all_predictions
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

optimal_perplexity = min(30, max(5, n_samples // 50))

tsne = TSNE(
    n_components=2,
    perplexity=optimal_perplexity,
    n_iter=1000,
    learning_rate=200,
    random_state=42,
    init='pca',
    metric='euclidean'
)

tsne_results = tsne.fit_transform(features_scaled)

ensemble_pred_class = np.argmax(ensemble_mean, axis=1)

fig1, ax1 = plt.subplots(figsize=(10,8))
cmap = plt.get_cmap("tab10", n_classes)
scatter1 = ax1.scatter(tsne_results[:,0], tsne_results[:,1],
                       c=ensemble_pred_class, cmap=cmap, alpha=0.7, s=20)
ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_ticks(range(n_classes))
cbar1.set_ticklabels(class_names)
cbar1.set_label('Predicted Class')
plt.savefig('raw_predictions_by_class.pdf', format='pdf', dpi=300)
plt.close()

original_reds = plt.get_cmap('Reds')(np.linspace(0,1,256))
darker_reds = original_reds[64:]
new_reds_cmap = ListedColormap(darker_reds)

fig2, ax2 = plt.subplots(figsize=(10,8))
scatter2 = ax2.scatter(tsne_results[:,0], tsne_results[:,1],
                       c=class_disagreement, cmap=new_reds_cmap, alpha=0.7, s=20,
                       vmin=0.3 * np.max(class_disagreement))
ax2.set_xlabel('t-SNE 1')
ax2.set_ylabel('t-SNE 2')
cbar2 = plt.colorbar(scatter2, ax=ax2)
cbar2.set_label('Conflict')
plt.savefig('raw_predictions_by_conflict.pdf', format='pdf', dpi=300)
plt.close()

class_std_per_sample = ensemble_std
max_conflict_class = np.argmax(class_std_per_sample, axis=1)
max_conflict_value = np.max(class_std_per_sample, axis=1)

avg_conflict_per_class = []
for i in range(n_classes):
    mask = max_conflict_class == i
    avg_conflict_per_class.append(np.mean(max_conflict_value[mask]) if np.sum(mask) > 0 else 0)

fig3, ax3 = plt.subplots(figsize=(10,6))
bars = ax3.bar(range(n_classes), avg_conflict_per_class,
               color=plt.cm.tab10(np.linspace(0,1,n_classes)))
ax3.set_xlabel('Class')
ax3.set_ylabel('Average Conflict')
ax3.set_xticks(range(n_classes))
ax3.set_xticklabels(class_names, rotation=45, ha='right')

for i, bar in enumerate(bars):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.savefig('average_conflict_by_class.pdf', format='pdf', dpi=300)
plt.close()

print("All three plots saved as PDF files.")
print("1. raw_predictions_by_class.pdf")
print("2. raw_predictions_by_conflict.pdf")
print("3. average_conflict_by_class.pdf")


Step 1: Normalizing predictions...
Step 2: Creating feature matrix...
Step 3: Preprocessing features...
Step 4: Processing raw predictions for t-SNE...
Creating Plot 1: Raw predictions colored by predicted class...
Creating Plot 2: Raw predictions colored by conflict...
Calculating conflict data for bar chart...
Creating Plot 3: Average conflict by class...
All three plots saved as separate PDF files:
1. raw_predictions_by_class.pdf
2. raw_predictions_by_conflict.pdf
3. average_conflict_by_class.pdf
